In [ ]:
import pandas as pd
import numpy as np
import math
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import kaleido
import re
import ast
import category_encoders as ce

from IPython.display import Image

from typing import List

Задача проекта: разработать сервис для предсказания стоимости домов на основе истории предложений.
Так как нам необходимо предсказать числовое значение, то нам необходимо решить задачу регрессии.

Для начала изучим исходные данные.

In [ ]:
df = pd.read_csv('data/data.csv')
df.head()

В полученном датасете имеем следующие поля: \

status - текущий статус собственности \
private pool - наличие собственного бассейна \
property type - тип собственности \
street - улица и номер дома \
baths - количество ванных комнат \
homeFacts - дополнительная информация о собственности \
fireplace - информация о камине \
city - название города \
schools - информация о близлежащих школах \
sqft - площадь собственности \
zipcode - почтовый индекс \
beds - количество спальных мест \
state - штат \
stories - количество этажей \
mls-id - идентификационный номер члена MLS \
PrivatePool - наличие собственного бассейна \
MlsId - идентификационный номер члена MLS \
target - стоимость собственности

In [ ]:
df.info()

In [ ]:
# удалим полные дубликаты строк
df = df.drop_duplicates()

В датасете достаточно много пропусков. Некоторые столбцы дублируют друг друга. \
Сначала проведем преобразование строк и выделим новые признаки.

##### Преобразуем столбец с целевым показателем.

In [ ]:
# удаляем строки в которых целевой показатель отсутствует
df = df.drop(df[df['target'].isna()].index)

In [ ]:
# найдем все строки в которых целевой показатель не является числом
df[~df['target'].str.isnumeric()]['target']

In [ ]:
# уберем знак долара и запятую из целевого показателя для того, чтобы можно было его
# преобразовать
df['target'] = df['target'].apply(lambda target: target.replace('$', '')\
                                                       .replace(',', ''))

In [ ]:
# еще раз проверим остались ли строки не являющиеся числом в столбце
df[~df['target'].str.isnumeric()]['target']

In [ ]:
# уберем знак плюса из строки с целевым показателем
# так как непонятно, что именно он может означать в датасете
df['target'] = df['target'].apply(lambda target: target.replace('+', ''))

In [ ]:
# проверим строки в которых содержится подстрока \mo
df[df['target'].str.contains('/mo')]

Судя по всему в данном датасете также содержится информация о собственности домов сдающихся в аренду. Но так как нам необходимо построить модель которая будет предсказывать стоимость домов, а не аренду, удалим данные строки из датасета

In [ ]:
df = df.drop(df[df['target'].str.contains('/mo')].index)

In [ ]:
# еще раз сделаем проверку, что в датасете не осталось строк, которые не являются числом
df[~df['target'].str.isnumeric()]['target']

In [ ]:
# преобразуем столбец к числовому виду
df['target'] = df['target'].astype(float)

##### Преобразуем столбец со статусом собственности.

In [ ]:
# посмотрим на значения в столбце
df['status'].value_counts()

Похоже, что некоторые значения дублируются в разном регистре. Необходимо это учесть при преобразовании.

In [ ]:
# посмотрим на значения в столбце, игнорируя регистр
df['status'].str.lower().value_counts().head(10)

Большая часть собственности имеет статус 'for sale' или 'active'. Оставим эти два статуса, а остальные сгруппируем в один.

In [ ]:
def group_status(status: str):
    if (status is not np.NaN and status.lower() in ['for sale', 'active']):
        return status.lower()
    return 'other'

In [ ]:
df['status'] = df['status'].apply(group_status)

In [ ]:
# проверим итоговый результат преобразования столбца
df['status'].value_counts()

##### Преобразуем столбец с информацией о наличии бассейна

Датасет содержит два столбца с информацией о наличии бассейна. Cоздадим новый столбец на основе двух существующих столбцов.

In [ ]:
# проверим содержимое полей
print(df['private pool'].str.lower().value_counts())
print(df['PrivatePool'].str.lower().value_counts())

In [ ]:
# функция для определения наличия бассейна
def fill_pool_data(df: pd.DataFrame):
    if (df['PrivatePool'] is not np.NaN and df['PrivatePool'].lower() == 'yes'):
        return 1
    if (df['private pool'] is not np.NaN and df['private pool'].lower() == 'yes'):
        return 1
    return 0

In [ ]:
df['private_pool'] = df.apply(fill_pool_data, axis=1)

In [ ]:
# удаляем столбцы, которые больше не нужны
df = df.drop(['PrivatePool', 'private pool'], axis=1)

##### Преобразуем столбец с типом собственности.

In [ ]:
# посмотрим на значения в столбце
df['propertyType'].value_counts()

In [ ]:
# посмотрим на 15 самых популярных значения
df['propertyType'].value_counts().head(15)

Похоже часть значений дублируется с разным написанием. Сгруппируем значения самых популярных типов.

In [ ]:
def rename_property_type(type: str):
    if type is np.NaN:
        return np.NaN
    
    type = type.lower()
    if ('single' in type and ('family' in type or 'detached' in type)):
        return 'single-family home'
    if ('multi' in type and 'family' in type):
        return 'multi-family home'
    if ('lot' in type or 'land' in type):
        return 'land'
    return type

In [ ]:
df['propertyType'] = df['propertyType'].apply(rename_property_type)

In [ ]:
# посмотрим на результат группировки
df['propertyType'].value_counts().head(15)

In [ ]:
# заменим пропуски на самое популярное значение
property_type_fill_string = df['propertyType'].value_counts().head(1).index[0]
df['propertyType'] = df['propertyType'].fillna(property_type_fill_string)

In [ ]:
df['propertyType'].value_counts().head(10)

Оставим 5 самых популярных значения остальные сгруппируем в одно

In [ ]:
def group_property_type(type: str):
    if (type in ['single-family home', 'condo', 'land', 'townhouse', 'multi-family home']):
        return type
    return 'other'

In [ ]:
df['propertyType'] = df['propertyType'].apply(group_property_type)

In [ ]:
# посмотрим на результат
df['propertyType'].value_counts()

##### Преобразуем столбец с улицей

In [ ]:
# проверим количество уникальных улиц
df['street'].value_counts().head(15)

В столбце слишком много уникальных значений. Скорее всего из-за того, что кроме улицы, значение также содержит номер дома. Уберем номер дома, а так же сгруппируем неисвезтные адреса в одно значение.

In [ ]:
def rename_street(street: str):
    
    if street is np.NaN:
        return 'unknown'
    
    street = street.lower()
    
    if ('undisclosed' in street \
        or 'not disclosed' in street \
        or 'not available' in street \
        or 'unknown' in street):
        
        return 'unknown'
    
    splited_street = street.split()
    
    if splited_street[0].isnumeric():
        return ' '.join(splited_street[1:])
    
    return street

In [ ]:
# проверим сколько остается значений после применения функции
df['street'].apply(rename_street).value_counts()

Уникальных значений в столбце даже после удаления номера дома все равно слишком много. Удалим столбец.

In [ ]:
df = df.drop('street', axis=1)

##### Преобразуем столбец с количеством ванных комнат

In [ ]:
# проверим уникальные значения в столбце
df['baths'].value_counts()

Преобразуем значения в числовой тип.

In [ ]:
def rename_bath(bath: str):
    if bath is np.NaN or bath in ['', '~', 'Sq. Ft.']:
        return np.NaN

    bath = bath.replace(',', '')
    
    match = re.search('[0-9]+\.?[0-9]*', bath)
    if not match:
        return np.NaN
    
    return match.group().strip()

In [ ]:
df['baths'] = df['baths'].apply(rename_bath).astype(float)
df['baths'] = df['baths'].round()

##### Преобразуем столбец с данными о площади

In [ ]:
df['sqft'].value_counts()

In [ ]:
def transform_sqft(sqft: str):
    if sqft is np.NaN or '--' in sqft:
        return np.NaN
    
    if type(sqft) is float:
        return sqft
    
    sqft = sqft.replace(',', '')
    match = re.search('[0-9]+\.?[0-9]*', sqft)
    if match:
        sqft = match.group()
    
    return sqft
    

In [ ]:
df['sqft'] = df['sqft'].apply(transform_sqft).astype(float)

##### Проведем преобразование данных о доме.

In [ ]:
# посмотрим, что хранится в этом признаке
df['homeFacts'].head()

Похоже, что в столбце хранится словарь с данными о доме. Посмотрим какие данные можно найти в данном словаре.

In [ ]:
keys = set()

def get_keyset_from_dict(dict_string: str):
    d = ast.literal_eval(dict_string)
    keys.update(d.keys())
    
df['homeFacts'].apply(get_keyset_from_dict)

print(keys)

Словарь содержит только один ключ, в котором хранится массив данных, которые тоже представляют собой словарь. Проверим какие типы данных встречаются у домов

In [ ]:
labels = set()

def get_labels_from_home_facts(dict_string: str):
    d = ast.literal_eval(dict_string)
    facts = d['atAGlanceFacts']
    for fact in facts:
        labels.add(fact['factLabel'])
    
df['homeFacts'].apply(get_labels_from_home_facts)

print(labels)

Итак в словаре хранятся данные о годе последней реконструкции дома, паркинге, стоимости квадратного фута, обогреве, охлаждении, размере участка и годе постройки. Извлечем эти данные в отдельные столбцы.

In [ ]:
def transform_home_facts(dict_string: str):
    new_home_facts = dict()
    d = ast.literal_eval(dict_string)
    facts = d['atAGlanceFacts']
    for fact in facts:
        new_home_facts[fact['factLabel']] = fact['factValue']
    
    return new_home_facts

In [ ]:
# преобразуем структуры, чтобы было легчи извлекать значения
df['transformed_home_facts'] = df['homeFacts'].apply(transform_home_facts)

In [ ]:
# разобъем словарь на отдельные признаки
df['remodeled_year'] = df['transformed_home_facts'].apply(lambda d: d['Remodeled year'])
df['parking'] = df['transformed_home_facts'].apply(lambda d: d['Parking'])
df['price_for_sqft'] = df['transformed_home_facts'].apply(lambda d: d['Price/sqft'])
df['heating'] = df['transformed_home_facts'].apply(lambda d: d['Heating'])
df['lot_size'] = df['transformed_home_facts'].apply(lambda d: d['lotsize'])
df['cooling'] = df['transformed_home_facts'].apply(lambda d: d['Cooling'])
df['year_built'] = df['transformed_home_facts'].apply(lambda d: d['Year built'])

In [ ]:
# уберем ненужные теперь столбцы
df = df.drop(['homeFacts', 'transformed_home_facts'], axis=1)

Преобразуем созданные признаки.

In [ ]:
df['year_built'].value_counts()

In [ ]:
df['year_built'] = df['year_built'].apply(lambda year: np.NaN if year is None or year == '' or year == 'No Data' else year).astype(float)

In [ ]:
df['remodeled_year'].value_counts()

In [ ]:
df['remodeled_year'] = df['remodeled_year'].apply(lambda year: 0 if year is None or year == '' else year).astype(int)

In [ ]:
df['parking'].value_counts().head(15)

In [ ]:
def transform_parking(parking):
    if parking is np.NaN or parking is None or parking == '':
        return np.NaN
    
    if parking.lower() in ['no data', 'none']:
        return '0'
    
    if 'Garage' in parking or 'Carport' in parking:
        return '1'
    
    match = re.search('[0-9]+', parking)
    if match:
        return match.group()
    
    return '1'

In [ ]:
df['parking'] = df['parking'].apply(transform_parking).astype(float)

In [ ]:
df['price_for_sqft']

In [ ]:
def transform_price_for_sqft(price):
    if price is np.NaN or price is None or price == 'No Info' or price == 'No Data' or price == '' or price == 'Contact manager':
        return 0
    
    price = price.replace('$', '')
    price = price.replace('/sqft', '')
    price = price.replace(' / Sq. Ft.', '')
    price = price.replace(',', '')
    
    
    return price

In [ ]:
df['price_for_sqft'] = df['price_for_sqft'].apply(transform_price_for_sqft).astype(float)

In [ ]:
# проверим нет ли в данном столбце утечки данных
df['price_for_sqft'] * df['sqft'].apply(lambda x: 0 if x is np.NaN else x) / df['target'] * 100 - 100

Судя по ввсему данный столбец может быть использован для вывода целевого значения. Необходимо его удалить.

In [ ]:
df = df.drop('price_for_sqft', axis=1)

In [ ]:
df['heating'].str.lower().value_counts().head(15)

In [ ]:
def transform_heating(heating: str):
    if heating is np.NaN or heating is None or heating.lower() in ['', 'no data']:
        return np.NaN
    
    heating = heating.lower()
    if 'forced' in heating:
        return 'forced'
    
    if 'central' in heating:
        return 'central'
    
    return 'other'
    

In [ ]:
df['heating'] = df['heating'].apply(transform_heating)

In [ ]:
df['lot_size'].str.lower().value_counts()

In [ ]:
def transform_lot_size(lot_size:str):
    sqft_in_acre = 43560
    
    if lot_size is np.NaN or lot_size is None or lot_size.lower() in ['', 'no data', '—', '-- sqft lot']:
        return np.NaN
    
    lot_size = lot_size.lower()
    lot_size = lot_size.replace(',', '')
    
    match = re.search('[0-9]+\.?[0-9]*', lot_size)
    if match:
        lot_size_number = float(match.group())
        if 'acres' in lot_size:
            lot_size_number *= sqft_in_acre
        return lot_size_number
    
    return lot_size

In [ ]:
df['lot_size'] = df['lot_size'].apply(transform_lot_size).astype(float)

In [ ]:
df['cooling'].str.lower().value_counts().head(15)

In [ ]:
def transform_cooling(cooling):
    if cooling is np.NaN or cooling is None or cooling.lower() in ['no data', 'none', '']:
        return np.NaN
    
    cooling = cooling.lower()
    
    if 'central' in cooling:
        return 'central'
    
    return 'other'

In [ ]:
df['cooling'] = df['cooling'].apply(transform_cooling)

##### Преобразуем столбец с данными о наличии камина

In [ ]:
# посмотрим на уникальные значения в столбце
df['fireplace'].value_counts()

Похоже, что в поле содержится разнородная информация. Где-то указано просто наличие камина, где-то количество, где-то описание. Попробуем вытащить информацию о наличии камина.

In [ ]:
def get_fireplace_number(fireplace: str):
    if fireplace is np.NaN:
        return np.NaN
    
    fireplace_keywords = ['yes', 'fireplace', 'one', 
                          '1', 'two', '2', 'three', '3', 'gas', 
                          'electric logs', '4', '4+', '5', '6',
                          '7', '8', '9', 'wood', 'frplc', 'electric', 
                          'living room', 'familyrm', 'great room', 'family room']
    fireplace = fireplace.lower()
    if fireplace == 'not applicable' or fireplace == '0' or fireplace == 'no':
        return 'no'
    for keyword in fireplace_keywords:
        if keyword in fireplace:
            return 'yes'

    return np.NaN

In [ ]:
df['fireplace'] = df['fireplace'].apply(get_fireplace_number)

##### Преобразуем столбец с данными о городах

In [ ]:
df['city'].str.lower().value_counts()

In [ ]:
# колонка содержит очень много уникальных значений
# посмотрим на процентное распределение городов
(df['city'].str.lower().value_counts() / df.shape[0] * 100).head(10)

In [ ]:
(df['city'].str.lower().value_counts() / df.shape[0] * 100).head(255).sum()

При дальнейшем кодировании, при использовании бинарного кодирования можно будет с помощью 8 новых столбцов описать 88 процентов данных.

In [ ]:
cities = df['city'].str.lower().value_counts().head(254).index
def transofrm_city(city):
    if city is np.NaN:
        return np.NaN
    
    city = city.lower()
    if city in cities:
        return city
    
    return 'other'

In [ ]:
df['city'] = df['city'].apply(transofrm_city)

##### Преобразуем столбец с данными о школах

In [ ]:
# посмотрим на значения в столбце
df['schools']

Похоже, что поле содержит сложную структуру данных, представляющую собой массив со словарями. Попробуем извлечь данные из структуры.

In [ ]:
df['schools'] = df['schools'].apply(lambda schools: ast.literal_eval(schools))
df['schools_rating'] = df['schools'].apply(lambda schools: schools[0]['rating'])
df['schools_distance'] = df['schools'].apply(lambda schools: schools[0]['data']['Distance'])
df['schools_grades'] = df['schools'].apply(lambda schools: schools[0]['data']['Grades'])
df['schools_name'] = df['schools'].apply(lambda schools: schools[0]['name'])

In [ ]:
df = df.drop('schools', axis=1)

In [ ]:
df['schools_rating']

Подсчитаем средний рейтинг школ у которых рейтинг заполнен.

In [ ]:
def transform_schools_rating(ratings:List[str]):
    transformed_rating = []
    for rating in ratings:
        if '/' in rating:
            rating = rating.split('/')[0]
            
        if rating.lower() not in ['nr', 'none', 'na', '']:
            transformed_rating.append(int(rating))
            
    return transformed_rating
        

In [ ]:
df['schools_rating'] = df['schools_rating'].apply(transform_schools_rating)

In [ ]:
def calculate_average_schools_rating(ratings: List[int]):
    if ratings == []:
        return np.NaN
    return sum(ratings) / len(ratings)

In [ ]:
df['schools_rating'] = df['schools_rating'].apply(calculate_average_schools_rating)

In [ ]:
df['schools_distance']

Посчитаем среднее расстояние до школ.

In [ ]:
def transform_schools_distance(distances:List[str]):
    transformed_distances = []
    for distance in distances:
        distance = distance.replace('mi', '').strip()
        distance = float(distance)
        transformed_distances.append(distance)
        
    return transformed_distances

In [ ]:
df['schools_distance'] = df['schools_distance'].apply(transform_schools_distance)

In [ ]:
def average_distance(distances:List[float]):
    if distances == []:
        return np.NaN
    
    return sum(distances) / len(distances)

In [ ]:
df['schools_distance'] = df['schools_distance'].apply(average_distance)

In [ ]:
# удалим столбец с названиями школ и классами
df = df.drop(['schools_name', 'schools_grades'], axis=1)

##### Преобразуем столбец с данными о количестве спальных мест

In [ ]:
df['beds'].value_counts()

In [ ]:
def transform_beds(beds):
    if beds is np.NaN:
        return np.NaN
    
    beds = beds.lower()
    
    if 'acre' in beds or 'sqft' in beds or 'bath' in beds or '--' in beds or ' ' in beds:
        return np.NaN
    
    if ',' in beds:
        beds_split = beds.split(',')
        for part in beds_split:
            if 'bedroom' in part:
                match = re.search('[0-9]+', part)
                if match:
                    return match.group()
                
    match = re.search('[0-9]+', beds)
    if match:
        return match.group()
    
    return beds

In [ ]:
df['beds'] = df['beds'].apply(transform_beds).astype(float)

##### Преобразуем столбец с количеством этажей

In [ ]:
def get_stories_number(stories: str):
    if stories is np.NaN:
        return stories
    
    if type(stories) is float:
        return stories
    
    if stories.replace('.', '').isnumeric():
        return float(stories)
    
    stories = stories.lower()
    
    if 'one' in stories or '1' in stories:
        return 1.0
    
    if 'two' in stories or '2' in stories:
        return 2.0
    
    if 'three' in stories or '3' in stories:
        return 3.0
    
    if 'four' in stories or '4' in stories:
        return 4.0
    
    if 'five' in stories or '5' in stories:
        return 5.0
    
    if 'six' in stories or '6' in stories:
        return 6.0
    
    # в случае если значение в строке не удается преобразовать возвращаем пустое значение,
    # чтобы в дальнейшем его заменить
    return np.NaN 

In [ ]:
df['stories'] = df['stories'].apply(get_stories_number)

##### Проверим поля с id члена MLS

In [ ]:
df['mls-id'].str.lower().value_counts()

In [ ]:
df['MlsId'].str.lower().value_counts()

Поле содержит слишком много уникальных значений. Удалим его.

In [ ]:
df = df.drop(['mls-id', 'MlsId'], axis=1)

In [ ]:
df['zipcode'].str.lower().value_counts()

Поле содержит слишком много уникальных значений. Удалим его.

In [ ]:
df = df.drop('zipcode', axis=1)

### Разберемся с пропусками в данных.

In [ ]:
df.info()

In [ ]:
# заполним пропуски в атрибуте с количеством ванн медианными значениями для каждого типа собственности
baths_by_property_type = df.groupby('propertyType')['baths'].median()
def fill_baths(row: pd.Series):
    if np.isnan(row['baths']):
        return baths_by_property_type[row['propertyType']]
    
    return row['baths']

In [ ]:
df['baths'] = df.apply(fill_baths, axis=1)

In [ ]:
# атрибут с информацией о наличии камина содержит слишком много пропусков. Его необходимо удалить.
df = df.drop('fireplace', axis=1)

In [ ]:
most_frequent_city = df[df['city'] != 'other']['city'].value_counts().index[0]
df['city'] = df['city'].fillna(most_frequent_city)

In [ ]:
# заполним пропуски в атрибуте с площадью собственности медианными значениями для каждого типа собственности
sqft_by_property_type = df.groupby('propertyType')['sqft'].median()
def fill_sqft(row: pd.Series):
    if np.isnan(row['sqft']):
        return sqft_by_property_type[row['propertyType']]
    
    return row['sqft']

In [ ]:
df['sqft'] = df.apply(fill_sqft, axis=1)

In [ ]:
# заполним пропуски в атрибуте с количеством спальных мест медианными значениями для каждого типа собственности
beds_by_property_type = df.groupby('propertyType')['beds'].median()
def fill_beds(row: pd.Series):
    if np.isnan(row['beds']):
        return beds_by_property_type[row['propertyType']]
    
    return row['beds']

In [ ]:
df['beds'] = df.apply(fill_beds, axis=1)

In [ ]:
# заполним пропуски в атрибуте с количеством этажей медианными значениями для каждого типа собственности
stories_by_property_type = df.groupby('propertyType')['stories'].median()
def fill_stories(row: pd.Series):
    if np.isnan(row['stories']):
        return stories_by_property_type[row['propertyType']]
    
    return row['stories']

In [ ]:
df['stories'] = df.apply(fill_stories, axis=1)

In [ ]:
# заполним пропуски в атрибуте с количество парковочных мест медианными значениями для каждого типа собственности в основных городах
parking_by_property_type = df.groupby(['propertyType'])['parking'].median()
def fill_parking(row: pd.Series):
    if np.isnan(row['parking']):
        return parking_by_property_type[row['propertyType']]
    
    return row['parking']

In [ ]:
df['parking'] = df.apply(fill_parking, axis=1)

In [ ]:
# заполним пропуски в атрибуте с типом обогрева значениями моды для каждого типа собственности
heating_by_property_type = df.groupby('propertyType')['heating'].agg(pd.Series.mode)
def fill_heating(row: pd.Series):
    if row['heating'] is np.NaN:
        return heating_by_property_type[row['propertyType']]
    
    return row['heating']

In [ ]:
df['heating'] = df.apply(fill_heating, axis=1)

In [ ]:
# заполним пропуски в атрибуте с размером участка медианными значениями для каждого типа собственности
lot_size_by_property_type = df.groupby(['propertyType'])['lot_size'].median()
def fill_lot_size(row: pd.Series):
    if np.isnan(row['lot_size']):
        return lot_size_by_property_type[row['propertyType']]
    
    return row['lot_size']

In [ ]:
df['lot_size'] = df.apply(fill_lot_size, axis=1)

In [ ]:
# заполним пропуски в атрибуте с типом охлаждения значениями моды для каждого типа собственности
cooling_by_property_type = df.groupby('propertyType')['cooling'].agg(pd.Series.mode)
def fill_cooling(row: pd.Series):
    if row['cooling'] is np.NaN:
        return cooling_by_property_type[row['propertyType']]
    
    return row['cooling']

In [ ]:
df['cooling'] = df.apply(fill_cooling, axis=1)

In [ ]:
# заполним пропуски в атрибуте с годом постройки медианными значениями в основных городах
year_built_by_city = df.groupby(['city'])['year_built'].median()
def fill_year_built(row: pd.Series):
    if np.isnan(row['year_built']):
        return year_built_by_city[row['city']]
    
    return row['year_built']

In [ ]:
df['year_built'] = df.apply(fill_year_built, axis=1)

In [ ]:
# заполним пропуски в атрибуте рейтингом школ медианными значениями в основных городах
schools_rating_by_city = df.groupby(['city'])['schools_rating'].median()
def fill_schools_rating(row: pd.Series):
    if np.isnan(row['schools_rating']):
        return schools_rating_by_city[row['city']]
    
    return row['schools_rating']

In [ ]:
df['schools_rating'] = df.apply(fill_schools_rating, axis=1)

In [ ]:
# заполним пропуски в атрибуте с расстоянием до школ медианными значениями в основных городах
schools_distance_by_city = df.groupby(['city'])['schools_distance'].median()
def fill_schools_distance(row: pd.Series):
    if np.isnan(row['schools_distance']):
        return schools_distance_by_city[row['city']]
    
    return row['schools_distance']

In [ ]:
df['schools_distance'] = df.apply(fill_schools_distance, axis=1)

### Разберемся с выбросами в данных.

In [ ]:
fig = px.histogram(df, x='baths')
fig.write_image('images/baths_histogram.png')
Image(filename='images/baths_histogram.png') 

Большая часть значений сконцентрировано возле 0. На графике видно длинный хвост справа из отдельных точек. Уберем выбросы при помощи методы Тьюки.

In [ ]:
def outliers_iqr(data, feature, left=1.5, right=1.5):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    
    outliers_filter = ''
    cleaned_filter = ''
    
    if left > 0:
        outliers_filter += '(x < lower_bound)'
        cleaned_filter += '(x >= lower_bound)'
    
    if right > 0:
        
        if outliers_filter != '':
            outliers_filter += ' | '
        outliers_filter += '(x > upper_bound)'
        
        if cleaned_filter != '':
            cleaned_filter += ' & '
        cleaned_filter += '(x <= upper_bound)'
        
    outliers = data[eval(outliers_filter)]
    cleaned = data[eval(cleaned_filter)]
    
    return outliers, cleaned

In [ ]:
_, df_cleaned = outliers_iqr(df, 'baths', left=0, right=5)
fig = px.histogram(df_cleaned, x='baths')
fig.write_image('images/baths_cleaned_histogram.png')
Image(filename='images/baths_cleaned_histogram.png')

In [ ]:
fig = px.histogram(df_cleaned, x='sqft')
fig.write_image('images/sqft_histogram.png')
Image(filename='images/sqft_histogram.png')

In [ ]:
_, df_cleaned = outliers_iqr(df_cleaned, 'sqft', left=0, right=3)
fig = px.histogram(df_cleaned, x='sqft') 
fig.write_image('images/sqft_cleaned_histogram.png')
Image(filename='images/sqft_cleaned_histogram.png')

In [ ]:
# из атрибута год реконструкции уберем строки со значениями меньше года постройки
df_cleaned = df_cleaned[~((df_cleaned['remodeled_year'] > 0) & (df_cleaned['remodeled_year'] < df_cleaned['year_built']))]

In [ ]:
def transform_remodeled_year(row: pd.Series):
    if row['remodeled_year'] == 0:
        return 2022 - row['year_built']
    
    return 2022 - row['remodeled_year']

In [ ]:
df_cleaned['remodeled_year'] = df_cleaned.apply(transform_remodeled_year, axis=1)

In [ ]:
df_cleaned = df_cleaned[df_cleaned['year_built'] <= 2022]

In [ ]:
fig = px.box(df_cleaned, x='year_built')
fig.write_image('images/year_built_boxplot.png')
Image(filename='images/year_built_boxplot.png')

In [ ]:
_, df_cleaned = outliers_iqr(df_cleaned, 'year_built', left=1.5, right=0)
fig = px.box(df_cleaned, x='year_built')
fig.write_image('images/year_built_cleaned_boxplot.png')
Image(filename='images/year_built_cleaned_boxplot.png')

In [ ]:
fig = px.box(df_cleaned, x='parking')
fig.write_image('images/parking_boxplot.png')
Image(filename='images/parking_boxplot.png')

In [ ]:
df_cleaned['parking'].value_counts().head(15)

In [ ]:
# у большей части собственности 10 или менее парковок. Оставим только эти данные, остальные будем считать за выбросы
df_cleaned = df_cleaned[df_cleaned['parking'] <= 10]
fig = px.box(df_cleaned, x='parking')
fig.write_image('images/parking_cleaned_boxplot.png')
Image(filename='images/parking_cleaned_boxplot.png')

In [ ]:
fig = px.histogram(df_cleaned, x='lot_size')
fig.write_image('images/lot_size_histogram.png')
Image(filename='images/lot_size_histogram.png')

In [ ]:
df_cleaned = df_cleaned[df_cleaned['lot_size'] <= 1000000]
fig = px.histogram(df_cleaned, x='lot_size')
fig.write_image('images/lot_size_cleaned_histogram.png')
Image(filename='images/lot_size_cleaned_histogram.png')

In [ ]:
fig = px.histogram(df_cleaned, x='schools_distance')
fig.write_image('images/schools_distance_histogram.png')
Image(filename='images/schools_distance_histogram.png')

In [ ]:
_, df_cleaned = outliers_iqr(df_cleaned, 'schools_distance', left=0, right=6)
fig = px.histogram(df_cleaned, x='schools_distance')
fig.write_image('images/schools_distance_cleaned_histogram.png')
Image(filename='images/schools_distance_cleaned_histogram.png')

In [ ]:
fig = px.box(df_cleaned, x="stories")
fig.write_image('images/stories_boxplot.png')
Image(filename='images/stories_boxplot.png')

In [ ]:
_, df_cleaned = outliers_iqr(df_cleaned, 'stories', left=0, right=2)
fig = px.box(df_cleaned, x="stories")
fig.write_image('images/stories_cleaned_boxplot.png')
Image(filename='images/stories_cleaned_boxplot.png')

##### Необходимо закодировать данные по категориальным признакам

In [ ]:
df_cleaned.head(5)

In [ ]:
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df_cleaned[['status', 'propertyType', 'heating', 'cooling']])
df_cleaned = df_cleaned.merge(encoded, left_index=True, right_index=True)

In [ ]:
encoder = ce.BinaryEncoder()
encoded = encoder.fit_transform(df_cleaned[['city', 'state']])
df_cleaned = df_cleaned.merge(encoded, left_index=True, right_index=True)

In [ ]:
df_cleaned = df_cleaned.drop(['status', 'propertyType', 'heating', 'cooling', 'city', 'state'], axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(30,15)) 
sns.heatmap(df_cleaned.corr(), annot=True, ax=ax)

In [ ]:
# уберем признаки с сильной корреляцией
df_cleaned = df_cleaned.drop(['year_built', 'cooling_2'], axis=1)

In [ ]:
df_cleaned.to_csv('data/data_cleaned.csv', index=False)